ssh spsither@prodigy.bdrc.io

./home/spsither/staging/export_all.sh

tar -cJvf staging.tar staging/*.jsonl

In [ ]:
! scp spsither@prodigy.bdrc.io:/home/spsither/staging.tar data/

In [ ]:
! tar -xf data/staging.tar -C data/

In [ ]:
import pandas as pd
def read_jsonl(filename):
    df = pd.read_json(filename,lines=True)
    df = df.reset_index(drop=True)
    return df

In [ ]:
import os

jsonl_directory = "data/staging/"

files = os.listdir(jsonl_directory)

pattern = "jsonl"
files_matching_pattern = [file for file in files if file.endswith(pattern)]

In [ ]:
#all reviewed files 
reviewed_jsonl_files = []
for file in files_matching_pattern:
    if 'review' in file and not 'stt_second_review' in file:
        reviewed_jsonl_files.append(file)

In [ ]:
#all unreviewed files
transcribed_jsonl_files = []
for file in files_matching_pattern:
    if 'review' not in file and not 'stt_second_review' in file:
        transcribed_jsonl_files.append(file)

In [ ]:
#concatenate all the transcribed dataframes
temp = pd.DataFrame([])
from tqdm.auto import tqdm
for file in tqdm(transcribed_jsonl_files):
    df = read_jsonl(f"{jsonl_directory}/{file}")
    temp = pd.concat([temp,df],axis=0)
    trabscribed_df = temp.reset_index(drop=True)

In [88]:
#concatenate all the reviewed dataframes
from tqdm.auto import tqdm
temp = pd.DataFrame([])
for file in tqdm(reviewed_jsonl_files):
    df = read_jsonl(f"{jsonl_directory}/{file}")
    temp = pd.concat([temp,df],axis=0)
    reviewed_df = temp.reset_index(drop=True)

100%|██████████| 25/25 [00:14<00:00,  1.70it/s]


In [89]:
finale_df = read_jsonl(f"{jsonl_directory}/stt_second_review.jsonl")

In [90]:
audio_extensions = ['.mp3', '.wav', '.MP3', '.WAV']

In [91]:
for audio_extension in audio_extensions:
    trabscribed_df['id'] = trabscribed_df['id'].str.replace(audio_extension, '')
    reviewed_df['id'] = reviewed_df['id'].str.replace(audio_extension, '')
    finale_df['id'] = finale_df['id'].str.replace(audio_extension, '')

In [93]:
trabscribed_df['id'].fillna(trabscribed_df['text'], inplace=True)
reviewed_df['id'].fillna(reviewed_df['text'], inplace=True)
finale_df['id'].fillna(finale_df['text'], inplace=True)

In [95]:
len(trabscribed_df),len(reviewed_df),len(finale_df)

(515579, 468714, 429954)

In [96]:
len(set(trabscribed_df['id'])), len(set(reviewed_df['id'])), len(set(finale_df['id']))

(446421, 425269, 369180)

In [97]:
100 - len(set(trabscribed_df['id'])) / len(trabscribed_df) * 100

13.413657266878602

In [98]:
trabscribed_df.drop_duplicates(subset='id', keep="first", inplace=True)
reviewed_df.drop_duplicates(subset='id', keep="first", inplace=True)
finale_df.drop_duplicates(subset='id', keep="first", inplace=True)

In [99]:
len(trabscribed_df),len(reviewed_df),len(finale_df)

(446421, 425269, 369180)

In [100]:
trabscribed_df = trabscribed_df[trabscribed_df['answer'] == 'accept']
reviewed_df = reviewed_df[reviewed_df['answer'] == 'accept']
finale_df = finale_df[finale_df['answer'] == 'accept']

In [101]:
len(trabscribed_df),len(reviewed_df),len(finale_df)

(428033, 413044, 358839)

In [102]:
#left side intersection
intersection = reviewed_df.merge(finale_df, how='left', on='id')

In [103]:
intersection['transcript_y'].fillna(intersection['transcript_x'], inplace=True)


In [104]:
intersection = intersection[~intersection['transcript_y'].isna()]

In [111]:
intersection = intersection[['transcript_y', 'id']]

In [112]:
intersection['url'] = 'https://d38pmlk0v88drf.cloudfront.net/wav/' + intersection['id'] + '.wav'

In [113]:
intersection.shape

(409687, 3)

In [114]:
i = 11001
intersection.loc[i]['transcript_y'], intersection.loc[i,'url']

('དེ་བཞིན་གཤེགས་པའི་བཀྲ་ཤིས་ཤོག་ཅེས་བརྗོད་པ་དང་དེ་ནས་ཕུན་ཚོགས་དང་ཚེ་བརྟན་ལྷ་མོར་ཁ་བཏགས་རེ་གཡོག་སྐབས་བཤད་པ་རྒྱག་འདོད་ཀྱིས་མགྲིན་པར་ཟ་ཕྲུག་ལངས་ནའང་།',
 'https://d38pmlk0v88drf.cloudfront.net/wav/STT_AB00252_0176_1498899_to_1507608.wav')

In [115]:
intersection['dept'] = intersection['id'].str[:6]

In [116]:
intersection.groupby('dept').count()

,transcript_y,id,url
dept,,,
STT_AB,37316,37316,37316
STT_CS,33055,33055,33055
STT_NS,42836,42836,42836
STT_TT,296480,296480,296480


In [119]:
import pyewts

converter = pyewts.pyewts()

intersection['wylie'] = intersection['transcript_y'].apply(converter.toWylie)

# བ་བྦ་a་བ་

In [120]:
intersection.rename(columns={"id": "file_name", "transcript_y": "uni", "wylie": "wylie", "url": "url", "dept": "dept"}, inplace=True)

In [121]:
last_intersection = intersection[['file_name', 'uni', 'wylie', 'url', 'dept']]
last_intersection.head()

,file_name,uni,wylie,url,dept
0,STT_NS_M0016_0001_2556_to_9914,རྩོམ་རིགས་ཀི་རྒན་ལགས་གསར་བ་ཅིག་སླེབས་འདུག་བ། ག...,rtsom rigs ki rgan lags gsar ba cig slebs 'dug...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_NS
1,STT_NS_M0016_0012_92258_to_102135,ཁོ་རང་གི་ད་བརྩེ་དུང་ལབ་ཡ་དེ་ཕ་མ་གི་བརྩེ་དུང་ཞེ...,kho rang gi da brtse dung lab ya de pha ma gi ...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_NS
2,STT_NS_M0016_0054_410923_to_420735,རེད། ད་ཁོ་རང་ད་རྩེད་མོ་ཧང་སང་ཡ་ཡག་པོ་རྩེ་ན་ཡེ།...,red/_da kho rang da rtsed mo hang sang ya yag ...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_NS
3,STT_NS_M0016_0053_401110_to_410923,ཨོ། ཨེ་ནས་ཨོ་འདི། འདི་ངས་དཔེ་ཆོབ་ཚ་བོ་མཐོང་གི་...,o/_e nas o 'di/_'di ngas dpe chob tsha bo mtho...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_NS
4,STT_NS_M0016_0013_104633_to_109510,ཨོ། ག་རེ་ལབ་དགོས་རེད་ད། ང་ཚོས་ཕ་མ་དེ་ཚོ་ད་འདས་...,o/_ga re lab dgos red da/_nga tshos pha ma de ...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_NS


In [122]:
last_intersection.to_csv('02_prodigy.tsv', sep='\t', index=False)

In [124]:
last_intersection[last_intersection['file_name'] == 'STT_AB00001_0029_150584_to_151748']

322327    https://d38pmlk0v88drf.cloudfront.net/wav/STT_...
Name: url, dtype: object